# Fraud Detection and Likelihood of Success Project

## Main Goals:
I want to be able to identify suspicious or fraudulent projects happening within economic development projects administered by the ESD based on historical data.  This identification will come from data where funding is disproportionate to project scope or industry, high job creation claims but low actual job creation, and large discrepancies between expected and actual disbursements.

The second goal I have for this project is to create a model that will help the ESD determine which projects to fund.

## Data Exploration
Exploring the data to see what variables I can work with

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [41]:
df = pd.read_csv('DOEI_Fraud_data.csv')
df.head(2)

/var/folders/5m/n3lmdh1d5bl0t6d_1p3n_2wh0000gn/T/ipykernel_29527/1063969295.py:1: DtypeWarning: Columns (0,11,12,14,23,24,27,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('DOEI_Fraud_data.csv')


,EIN of the recipient,Lead Agency Name,Lead Agency Code,Quarter Number of the Submission,Project ID Number,Program through which the funding was awarded,Recipient Name,Name of project,Is the Award to a Public or Private Sector Entity?,Project Description,...,Project Hires (PTs),NYS MWBE Utilization Goal Applied,Current MWBE Utilization Rate,Is this a legacy project,Original Assistance Amount,Original Jobs Created (FTE),Original Jobs Retained (FTE),Original Total Public-Private Investment,Original Recipient,Contact
0,16-09119851,ESD,291910,Q3 of SFY 2024-2025,"123,581",Excelsior Jobs Program,Aakron Rule Corporation/ Wagner Pencil Company,Aakron Rule Excelsior,Private,"Tax Credit, Manufacturing, Western New York, A...",...,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,database@esd.ny.gov
1,14-1811177,ESD,291910,Q3 of SFY 2024-2025,"123,585",Regional Council Capital Fund,Schenectady Metroplex Development Authority,Robinson Block Capital,Public,"Grant, Public Administration, Capital District...",...,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,database@esd.ny.gov


In [43]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63364 entries, 0 to 63363
Data columns (total 60 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   EIN of the recipient                                                                      3340 non-null   object 
 1   Lead Agency Name                                                                          63364 non-null  object 
 2   Lead Agency Code                                                                          63364 non-null  int64  
 3   Quarter Number of the Submission                                                          63364 non-null  object 
 4   Project ID Number                                                                         63364 non-null  object 
 5   Program through which the funding was awarded        

In [62]:
df['If the project is a Member Item, the originating district'].value_counts()

If the project is a Member Item, the originating district
Senate District #48      16
Executive                11
Senate District #26      11
Senate District #34      10
Conference Wide (S)      10
                         ..
Senate District #46       1
Senate District #16       1
Senate District #27       1
Senate District #40       1
Assembly District #72     1
Name: count, Length: 69, dtype: int64

In [60]:
df_cleaned = df.drop(df.columns[0:5], axis = 1)
df_cleaned = df_cleaned.drop(['Project Description', 'Contact', 
                              'Penalties Applied?', 'Rationale for Termination',
                             'Reason for Non-Compliant Status', 'Street Address',
                             'Postal Code'], axis = 1)

#Convert all instances where industry isnt specified to Not Specified or "NS"
df_cleaned['Industry'].fillna('NS', inplace=True)

In [61]:
print(df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63364 entries, 0 to 63363
Data columns (total 48 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   Program through which the funding was awarded                                             63364 non-null  object 
 1   Recipient Name                                                                            63362 non-null  object 
 2   Name of project                                                                           63362 non-null  object 
 3   Is the Award to a Public or Private Sector Entity?                                        63364 non-null  object 
 4   Industry                                                                                  63364 non-null  object 
 5   If the project is a Member Item, the originating dist

### Selected Variables
Program through which the funding was awarded: Name of program or funding source that is funding the economic assistance from reporting agency.

Recipient Name: Name of organization receiving economic assistance from reporting agency.

Name of project: Name of project.

Is the Award to a Public or Private Sector Entity?: "Public" entity being any governmental entity, including but not limited to IDAs, cities, towns, villages, or counties. "Private" entity being any entity that does not fall under the definition of “Public” entity, including but not limited to private businesses and non-profit organizations.

Industry: Primary industry sector of the project, based upon the two-digit NAICS code for recipient.

If the project is a Member Item, the originating district: a project that is specifically allocated or sponsored by a government representative, such as a legislator or assembly member

Street Address: Street or building location of project; projects with multiple addresses either list a primary address or “Various”. Some exact locations are approximated.

County: County for location of project.

Postal Code: Zip code for location of project.

Economic Development Region: Region for location of project (derived from the 10 New York State Economic Development Regions).
economic_development_region

DOS: Start date of the contract ESD

End Date
DOS: Date that all financial assistance has been disbursed for the project. ESD: Date that all financial assistance has been disbursed for the project. HCR: Not applicable or not currently collected. NYPA: Date that all financial assistance has been disbursed for the project. NYSERDA: Date at which projects are completed.

Assistance Type
DOS: Type of assistance the project is receiving, defined as either loan, grant, or tax credit. ESD: Type of assistance the project is receiving, defined as either loan, grant, or tax credit. HCR: Not applicable or not currently collected. NYPA: Type of assistance the project is receiving, defined as either loan, grant, or tax credit. NYSERDA: NYSERDA’s funding type classification.

Assistance Amount
All: The total dollar value for the specific agency and project award. This does not include additional dollars awarded to the project from other state sources. - START-UP: The Start Date is not necessarily the beginning of the ten-year tax-free period or the date of disbursement. There is typically a lag in the disbursement time period.

Total Lead Agency Benefits Awarded
DOS: Total dollar amount of DOS assistance the project is receiving through the Downtown Revitalization Initiative ESD: Total amount of ESD assistance the project is receiving in dollars. - START-UP: The Start Date is not necessarily the beginning of the ten-year tax-free period or the date of disbursement. There is typically a lag in the disbursement time period. HCR: Amount of funding provided through the CDBG grant by HCR. NYPA: Total amount in dollars of the grant funding awarded by NYPA for the project described NYSERDA: Cumulative NYSERDA funding contracted through the most recent reporting period. Green Bank loans are omitted.

Disbursements to Date
All: The total dollars disbursed to the project, subject to a reporting lag.

Does this project include additional NYS benefits?
All: Indicates whether the project is receiving benefits outside of the reporting agency, program and award amount. This field can include other investments from the reporting agency (Yes/No).

If "YES" what is the additional awarding agency name?
If "Yes" to "Does the project include additional NYS benefits?", the name of the additional agency providing funding, if applicable.

Other State Agency Funding Program
All: The program through which the other agency is providing funding, if applicable.

Other State Agency Funding Awarded
All: Dollar amount of funding from other agencies, if applicable.

Is the Project also receiving benefits from a local Industrial Development Agency (IDA)?
All: Is the Project also receiving benefits from a local Industrial Development Agency (IDA) (Yes / No)

If applicable, from which IDA is the project also receiving benefits?
All: Name of the Industrial Development Agency, if applicable

If applicable, what is the net value of the tax benefits awarded by the IDA?
DOS: Calculated as net exemptions provided minus any PILOT payments made over the life of the IDA exemption, if applicable ESD: Calculated as the market value of all payments that would have been due minus any payments made. HCR: Not applicable or not currently collected. NYPA: n/a if not applicable NYSERDA: Not applicable or not currently collected.

Total NYS Investment
DOS: Total dollar amount of DOS and other known NYS agency investments ESD: Total dollar amount of the awarding agency and other known NYS agency investments. - START-UP: The Start Date is not necessarily the beginning of the ten-year tax-free period or the date of disbursement. There is typically a lag in the disbursement time period. HCR: The sum of HCR and other agency partner investment amounts. NYPA: Total amount in dollars, to include other NYS awards if known NYSERDA: Cumulative NYSERDA funding contracted through the most recent reporting period. Green Bank loans are omitted.


## Data Pre-Processing
